<div style="border: 10px solid #1976D2; border-radius: 15px; padding: 20px 20px 20px 20px; box-shadow: 3px 3px 10px rgba(0, 0, 0, 0.2); background-color: #f4f4f4;">
    <h2 style="font-size: 28px; font-weight: bold; color: #1976D2; text-align: center; border-bottom: 4px ridge #64B5F6; font-family: 'Georgia'; padding: 10px;">Training a Faster RCNN X101 Detector</h2>


In [1]:
# Detectron2 Installation Process
# cu101 must be installed as it is CUDA 10.1
!pip install -U torch==1.5 torchvision==0.6 -f https://download.pytorch.org/whl/cu101/torch_stable.html 
!pip install cython pyyaml==5.1
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version

Looking in links: https://download.pytorch.org/whl/cu101/torch_stable.html
ERROR: Could not find a version that satisfies the requirement torch==1.5 (from versions: 1.11.0, 1.12.0, 1.12.1, 1.13.0, 1.13.1, 2.0.0, 2.0.1, 2.1.0)
ERROR: No matching distribution found for torch==1.5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.2/274.2 kB 8.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ done
  Created wheel for pyyaml: filename=PyYAML-5.1-cp310-cp310-linux_x86_64.whl size=44091 sha256=691c02571ce5146fb7b89b380b2bd42b6c9bf6cecef5ba4950c05af62b43c882
  Stored in directory: /root/.cache/pip/wheels/70/83/31/975b737609aba39a4099d471d5684141c1fdc3404f97e7f68a
Successfully built pyyaml
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 6.0
    Uninstalling PyYAML-6.0:
      Successfully uninstalled PyYAML-6.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following

In [2]:
# detectron2 setup
!pip install detectron2==0.1.3 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.5/index.html

Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.5/index.html
ERROR: Could not find a version that satisfies the requirement detectron2==0.1.3 (from versions: none)
ERROR: No matching distribution found for detectron2==0.1.3


In [3]:
# Runtime can be restarted in this section
# detectron2 logger installation
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

ModuleNotFoundError: No module named 'detectron2'

In [ ]:
# common libraries should be imported
import numpy as np
import cv2
import random
from google.colab.patches import cv2_imshow

In [ ]:
# some common detectron2 features should be imported
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
from detectron2.data.catalog import DatasetCatalog

In [ ]:
#Importing Detectron2 Dataset
!curl -L "https://app.roboflow.com/ds/zi9LUNojmz?key=H5YnroNUmG" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

from detectron2.data.datasets import register_coco_instances
register_coco_instances("my_dataset_train", {}, "/content/train/_annotations.coco.json", "/content/train")
register_coco_instances("my_dataset_val", {}, "/content/valid/_annotations.coco.json", "/content/valid")
register_coco_instances("my_dataset_test", {}, "/content/test/_annotations.coco.json", "/content/test")

In [ ]:
#loading the training dataset
my_dataset_train_metadata = MetadataCatalog.get("my_dataset_train")
dataset_dicts = DatasetCatalog.get("my_dataset_train")

import random
from detectron2.utils.visualizer import Visualizer

for d in random.sample(dataset_dicts, 3):
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=my_dataset_train_metadata, scale=0.5)
    vis = visualizer.draw_dataset_dict(d)
    cv2_imshow(vis.get_image()[:, :, ::-1])



In [ ]:
# Training of Detectron2 Detector
from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator

class CocoTrainer(DefaultTrainer):

  @classmethod
  def build_evaluator(cls, cfg, dataset_name, output_folder=None):

    if output_folder is None:
        os.makedirs("coco_eval", exist_ok=True)
        output_folder = "coco_eval"

    return COCOEvaluator(dataset_name, cfg, False, output_folder)



In [ ]:
# CNN architecture should be selected from modelzoo section: https://github.com/facebookresearch/detectron2/blob/master/MODEL_ZOO.md#coco-object-detection-baselines

from detectron2.config import get_cfg
import os

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("my_dataset_train",)
cfg.DATASETS.TEST = ("my_dataset_val",)

cfg.DATALOADER.NUM_WORKERS = 4
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml")  # Model zoo dan eğitim başlatılmalı
cfg.SOLVER.IMS_PER_BATCH = 4
cfg.SOLVER.BASE_LR = 0.001

cfg.SOLVER.WARMUP_ITERS = 1000
cfg.SOLVER.MAX_ITER = 6000 #Modelin iterasyon sayısına göre overfit etmesi ya da başarının artma durumunun devam etme potansiyeline göre iterasyon sayısı ayarlanmalıdır
cfg.SOLVER.STEPS = (1000, 1500)
cfg.SOLVER.GAMMA = 0.05



cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 64
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 2 #sınıf sayısı + 1

cfg.TEST.EVAL_PERIOD = 500

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = CocoTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()



In [ ]:
# The performance of the model should be checked on the Tensorboard.
%load_ext tensorboard
%tensorboard --logdir output

#test evaluation
from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_test_loader
from detectron2.evaluation import COCOEvaluator, inference_on_dataset

cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_0004999.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.85
predictor = DefaultPredictor(cfg)
evaluator = COCOEvaluator("my_dataset_test", cfg, False, output_dir="./output/")
val_loader = build_detection_test_loader(cfg, "my_dataset_test")
inference_on_dataset(trainer.model, val_loader, evaluator)

In [ ]:
#Inference from Detectron2 registered weights
%ls ./output/

from detectron2.utils.visualizer import ColorMode
import glob

for imageName in glob.glob('/content/test/*jpg'):
  im = cv2.imread(imageName)
  outputs = predictor(im)
  v = Visualizer(im[:, :, ::-1],
                metadata=test_metadata, 
                scale=0.8
                 )
  out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
  cv2_imshow(out.get_image()[:, :, ::-1])

### YOLO v4 

In [ ]:

#Cloning and installation of #Darknet library
# clone darknet repo
!git clone https://github.com/AlexeyAB/darknet

from tensorflow.python.client import device_lib
device_lib.list_local_devices()

In [ ]:
# Change "makefile" to enable GPU and OPENCV
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile

# darknet setup
!make

# Load pre-trained YOLO v4 weights
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.weights

In [ ]:
# helper functions
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

In [ ]:
# uploading files
def upload():
  from google.colab import files
  uploaded = files.upload() 
  for name, data in uploaded.items():
    with open(name, 'wb') as f:
      f.write(data)
      print ('saved file', name)

# used for file download
def download(path):
  from google.colab import files
  files.download(path)

%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

# this step creates a symbolic link between “/content/gdrive/My\ Drive/” and “/mydrive” extensions
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

# return to the previous darknet folder to run the detection function
%cd darknet

<div style="border: 10px solid #1976D2; border-radius: 15px; padding: 20px 20px 20px 20px; box-shadow: 3px 3px 10px rgba(0, 0, 0, 0.2); background-color: #f4f4f4;">
    <h2 style="font-size: 28px; font-weight: bold; color: #1976D2; text-align: center; border-bottom: 4px ridge #64B5F6; font-family: 'Georgia'; padding: 10px;">Training a YOLOv4 Detector</h2>
    <p style="font-size: 16px; font-family: Arial, sans-serif; line-height: 1.6; padding: 10px;">
        <span style="font-size: 20px;">#</span> How to Train a Custom YOLOv4 Detector:
    </p>
    <p style="font-size: 16px; font-family: Arial, sans-serif; line-height: 1.6; margin-left: 20px; padding: 10px;">
        To create a custom YOLOv4 detector, the following components are essential:
    </p>
    <ul style="font-size: 16px; font-family: Arial, sans-serif; line-height: 1.6; padding: 10px; margin-left: 20px;">
        <li>Labeled Custom Dataset</li>
        <li>Custom .cfg file</li>
        <li>obj.data and obj.names files</li>
        <li>train.txt file (test.txt is optional here as well)</li>
    </ul>
    <p style="font-size: 16px; font-family: Arial, sans-serif; line-height: 1.6; padding: 10px;">
        The next step is to transfer your custom dataset to the cloud virtual machine:
    </p>
</div>


In [ ]:
!ls /mydrive/yolov4

# both datasets are copied to the root directory of the Colab virtual machine
!cp /mydrive/yolov4/obj.zip ../
!cp /mydrive/yolov4/test.zip ../

# unzip in virtual machine
!unzip ../obj.zip -d data/
!unzip ../test.zip -d data/

In [ ]:
#Educational Design
Step #u involves properly configuring your custom .cfg, obj.data, obj.names, train.txt and test.txt files.
#It is important to configure all these files extremely carefully, as typos or minor errors can cause major problems in special training.
#i) File with Cfg Extension
#Copy the yolov4.cfg file to your Google Drive by running the cell below. This will allow us to edit it in a text editor.

# download cfg to google drive and rename
!cp cfg/yolov4-custom.cfg /mydrive/yolov4/yolov4-obj.cfg

# to download to local machine (rename to yolov4-obj.cfg after downloading)
download('cfg/yolov4-custom.cfg')

<div style="border: 10px solid #1976D2; border-radius: 15px; padding: 20px 20px 20px 20px; box-shadow: 3px 3px 10px rgba(0, 0, 0, 0.2); background-color: #f4f4f4;">
    <h2 style="font-size: 28px; font-weight: bold; color: #1976D2; text-align: center; border-bottom: 4px ridge #64B5F6; font-family: 'Georgia'; padding: 10px;">Customizing Your Configuration</h2>
    <p style="font-size: 16px; font-family: Arial, sans-serif; line-height: 1.6; padding: 10px;">
        <span style="font-size: 20px;">⚙️</span> In this step, you'll fine-tune your object detector's .cfg file to match your specific requirements:
    </p>
    <p style="font-size: 16px; font-family: Arial, sans-serif; line-height: 1.6; margin-left: 20px; padding: 10px;">
        If you've saved the .cfg file to Google Drive, navigate to your Google Drive and double-click on the yolov4-obj.cfg file. Choose "Open with" and select Text Editor to use the built-in tool.
    </p>
    <p style="font-size: 16px; font-family: Arial, sans-serif; line-height: 1.6; margin-left: 20px; padding: 10px;">
        For optimal results, consider setting batch = 64 and subdivisions = 16. If you encounter any issues, increase subdivisions to 32.
    </p>
    <p style="font-size: 16px; font-family: Arial, sans-serif; line-height: 1.6; margin-left: 20px; padding: 10px;">
        Customize the rest of the .cfg file based on the number of classes you are training your object detector for. Update the classes value in the YOLO layers and filters value in the convolutional layers before the YOLO layers, adjusting to (number of classes + 5) * 3.
    </p>
    <p style="font-size: 16px; font-family: Arial, sans-serif; line-height: 1.6; padding: 10px;">
        Configure the following variables as follows:
    </p>
    <ul style="font-size: 16px; font-family: Arial, sans-serif; line-height: 1.6; padding: 10px; margin-left: 20px;">
        <li>width = 416</li>
        <li>height = 416 (these values can be multiples of 32, with 416 being the standard; you can occasionally increase it, e.g., 608, for improved results, though it may slow down training)</li>
        <li>max_batches = (# of classes) * 2000 (should not be less than 6000, e.g., 1, 2, and 3 classes should have 6000 iterations)</li>
        <li>steps = 80% of max_batches, 90% of max_batches</li>
        <li>filters = (number of classes + 5) * 3 (e.g., if you have 1 class, filters = 18; for 4 classes, filters = 27)</li>
    </ul>
    <p style="font-size: 16px; font-family: Arial, sans-serif; line-height: 1.6; margin-left: 20px; padding: 10px;">
        Once configured, upload your customized .cfg file back to your cloud environment, such as your cloud virtual machine.
    </p>
</div>


In [ ]:

!cp /mydrive/yolov4/yolov4-obj.cfg ./cfg

<div style="border: 10px solid #1976D2; border-radius: 15px; padding: 20px 20px 20px 20px; box-shadow: 3px 3px 10px rgba(0, 0, 0, 0.2); background-color: #f4f4f4;">
    <h2 style="font-size: 28px; font-weight: bold; color: #1976D2; text-align: center; border-bottom: 4px ridge #64B5F6; font-family: 'Georgia'; padding: 10px;">Setting Up obj.names and obj.data Files</h2>
    <p style="font-size: 16px; font-family: Arial, sans-serif; line-height: 1.6; padding: 10px;">
        <span style="font-size: 20px;">ii)</span> Create the obj.names and obj.data files to enable efficient model training and backup:
    </p>
    <p style="font-size: 16px; font-family: Arial, sans-serif; line-height: 1.6; margin-left: 20px; padding: 10px;">
        In a code or text editor, create a new file named obj.names. Each line in this file should correspond to a class name, in the same order as the classes in the vclasss.txt file created during the dataset creation step.
    </p>
    <p style="font-size: 16px; font-family: Arial, sans-serif; line-height: 1.6; margin-left: 20px; padding: 10px;">
        Additionally, create an obj.data file and populate it as follows:
    </p>
    <pre style="font-size: 16px; font-family: Arial, sans-serif; line-height: 1.6; margin-left: 20px; padding: 10px; background-color: #f9f9f9; border: 1px solid #ddd; border-radius: 5px; padding: 10px;">
classes = (number of classes)
train = data/train.txt
valid = data/test.txt
names = data/obj.names
backup = /content/drive/MyDrive/backup_folder/
    </pre>
    <p style="font-size: 16px; font-family: Arial, sans-serif; line-height: 1.6; margin-left: 20px; padding: 10px;">
        The "backup" path specifies where the model weights will be saved during training. Create a backup folder on your Google Drive platform and adjust the path accordingly.
    </p>
    <p style="font-size: 16px; font-family: Arial, sans-serif; line-height: 1.6; padding: 10px;">
        Finally, upload the obj.names and obj.data files to your cloud virtual machine.
    </p>
</div>


In [ ]:

!cp /mydrive/yolov4/obj.names ./data
!cp /mydrive/yolov4/obj.data  ./data

<div style="border: 10px solid #1976D2; border-radius: 15px; padding: 20px 20px 20px 20px; box-shadow: 3px 3px 10px rgba(0, 0, 0, 0.2); background-color: #f4f4f4;">
    <h2 style="font-size: 28px; font-weight: bold; color: #1976D2; text-align: center; border-bottom: 4px ridge #64B5F6; font-family: 'Georgia'; padding: 10px;">Creating train.txt and test.txt Files</h2>
    <p style="font-size: 16px; font-family: Arial, sans-serif; line-height: 1.6; padding: 10px;">
        <span style="font-size: 20px;">iii)</span> Before you commence training your custom detector, you need to prepare the essential configuration files, train.txt and test.txt, which contain the relevant extensions to all training and validation images:
    </p>
    <p style="font-size: 16px; font-family: Arial, sans-serif; line-height: 1.6; margin-left: 20px; padding: 10px;">
        These files hold the keys to your custom detector's training process. They list the paths to all training and validation images, respectively.
    </p>
    <p style="font-size: 16px; font-family: Arial, sans-serif; line-height: 1.6; margin-left: 20px; padding: 10px;">
        To create these files, upload the create_train.py and create_test.py script files from your Google Drive to your cloud virtual machine.
    </p>
</div>


In [ ]:
!cp /mydrive/yolov4/generate_train.py ./
!cp /mydrive/yolov4/generate_test.py ./

!python generate_train.py
!python generate_test.py

!ls data/

#Pre-trained weights are downloaded for learning transfer
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137

In [ ]:

#Detector's Training

# training of the special detector!
!./darknet detector train data/obj.data cfg/yolov4-obj.cfg yolov4.conv.137 -dont_show -map

#After the training, run the command below to get a table showing how your model has worked throughout the training process. A graph of the average loss versus iterations is shown. A loss of less than 2% is targeted for the model to be effective.
# show chart.png of how custom object detector did with training
imShow('chart.png')


#We can start the training from the last saved weights file so that there is no need to start over.
# Start the tutorial from where it was last saved
!./darknet detector train data/obj.data cfg/yolov4-obj.cfg /mydrive/yolov4/backup/yolov4-obj_last.weights -dont_show

#Checking the Mean Average Precision (mAP) Value
#If the tutorial wasn't run with the '-map- flag added, you can learn the map of the model after the tutorial. To see the mAP value for the file of that weight, run the following command on any of the weights saved from the training.!./darknet detector map data/obj.data cfg/yolov4-obj.cfg /mydrive/yolov4/backup/yolov4-obj_last.weights

#Custom Object Detector Running for Picture
# we need to set custom cfg file to test mode
%cd cfg
!sed -i 's/batch=64/batch=1/' yolov4-obj.cfg
!sed -i 's/subdivisions=16/subdivisions=1/' yolov4-obj.cfg
%cd ..

# custom detector is run with this command (image uploaded to google drive platform for testing)
!./darknet detector test data/obj.data cfg/yolov4-obj.cfg /mydrive/yolov4/backup/yolov4-obj_last.weights /mydrive/images/UAV8.jpg -thresh 0.3
imShow('predictions.jpg')

#Custom Object Detector Run for Video

!./darknet detector demo data/obj.data cfg/yolov4-obj.cfg /mydrive/yolov4/backup/yolov4-obj_last.weights -dont_show /mydrive/videos/UAV3.mp4 -i 0 -out_filename /mydrive/videos/results3.avi
